In [1]:
import sqlite3
import random
import networkx as nx
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import time



conn = sqlite3.connect("db_numprop-4_nestlim-100.db")
cursor = conn.cursor()

cursor.execute("SELECT * FROM data")
all_rows = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
formula_idx = column_names.index("formula")
category_idx = column_names.index("category")

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
from helpers import find_allowable_combinations, round_prediction, binary_to_bitlist, tree_to_formula, run_derivation_for_row

def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            nn.init.zeros_(m.bias)


class Net(nn.Module):
    def __init__(self, input_size, output_size):
        super(Net, self).__init__()
        self.ln1 = nn.Linear(input_size, 16)
        self.ln2 = nn.Linear(16, 16)
        self.ln3 = nn.Linear(16, output_size)

    def forward(self, x):
        x = 2 * x - 1
        x = x.to(device)
        x = F.relu(self.ln1(x))
        x = F.relu(self.ln2(x))
        x = self.ln3(x)
        return torch.sigmoid(x)


MODEL_CACHE = {}
def get_model(input_size, output_size):
    if output_size not in MODEL_CACHE:
        model = Net(input_size, output_size).to(device)
        MODEL_CACHE[output_size] = model
    model = MODEL_CACHE[output_size]
    model.apply(init_weights)  
    return model

# Compute the target vector for a given row and tree candidate
def compute_target(correct, input_row, nn_first_prediction, tree_candidate):
    assignments = tuple(input_row)
    candidate_list, non_terminal_count = find_allowable_combinations(tree_candidate, correct, assignments)

    if not candidate_list or non_terminal_count == 0:
        return None, 0

    candidate_vectors = [
        torch.tensor(
            [candidate.get(f"Z_{i}", 0) for i in range(non_terminal_count)],
            dtype=torch.float,
            device=device
        )
        for candidate in candidate_list
    ]

    if not candidate_vectors:
        return None, 0

    candidate_tensor = torch.stack(candidate_vectors)
    nn_pred = nn_first_prediction.view(1, -1).to(device)
    distances = torch.norm(candidate_tensor - nn_pred, dim=1)
    best_idx = torch.argmin(distances).item()
    return candidate_vectors[best_idx], non_terminal_count

# Improved train_on_truth_table function
def train_on_truth_table(nn_model, truth_table, bitlist, tree_candidate, max_iterations=5000):
    optimizer = torch.optim.Adam(nn_model.parameters(), lr=0.01)
    truth_tensor = torch.from_numpy(truth_table.astype(np.float32)).to(device)
    
    # First, get the output size and check if we can solve this candidate
    first_row = truth_tensor[0].unsqueeze(0)
    with torch.no_grad():
        dummy_pred = nn_model(first_row).squeeze(0)
    
    first_target, output_size = compute_target(bitlist[0], tuple(truth_table[0]), dummy_pred, tree_candidate)
    if first_target is None:
        return None
    
    # Check if all rows can be solved with this tree candidate
    all_solvable = True
    for i in range(len(truth_table)):
        with torch.no_grad():
            row_pred = nn_model(truth_tensor[i].unsqueeze(0)).squeeze(0)
        row_target, _ = compute_target(bitlist[i], tuple(truth_table[i]), row_pred, tree_candidate)
        if row_target is None:
            all_solvable = False
            break
    
    if not all_solvable:
        return None
        
    # Training loop - process one row at a time
    iterations_per_row = []
    max_iter = 1000
    
    for i, row in enumerate(truth_table):
        row_tensor = torch.from_numpy(row.astype(np.float32)).unsqueeze(0).to(device)
        correct = bitlist[i]
        
        training_iter = 0
        while training_iter < max_iter:
            # Forward pass
            pred = nn_model(row_tensor)
            pred_flat = pred.squeeze(0)
            
            # Get target for this prediction
            target, _ = compute_target(correct, tuple(row), pred_flat, tree_candidate)
            if target is None:
                return None
                
            # Check if we've converged
            pred_binary = round_prediction(pred_flat)
            if torch.equal(pred_binary, target):
                break
                
            # Compute loss and update
            loss = F.binary_cross_entropy(pred, target.unsqueeze(0))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            training_iter += 1
            
        if training_iter == max_iter:
            return float('inf')  # This candidate is too difficult to train
            
        iterations_per_row.append(training_iter)
    
    # Return the total number of iterations
    return sum(iterations_per_row)

def evaluate_candidate_options(current_options, truth_table, final_formula, bitlist):
    candidate_iterations = {}
    
    for idx, tree_cand in current_options.items():
        cand_str = tree_to_formula(tree_cand)
        
        # Skip fully instantiated formulas or check for exact match
        if 'Z' not in cand_str:
            if cand_str == final_formula:
                return cand_str  # Found exact match
            continue
                
        # Check if this candidate is viable
        first_assignment = tuple(truth_table[0])
        candidate_list, non_terminal_count = find_allowable_combinations(tree_cand, bitlist[0], first_assignment)
        
        if non_terminal_count == 0 or not candidate_list:
            continue  # Skip non valid, maybe implement check for good formula here
            
        # Get a model with the right output size
        nn_model = get_model(truth_table.shape[1], non_terminal_count)
            
        # Train and get iterations
        iters = train_on_truth_table(nn_model, truth_table, bitlist, tree_cand)
        
        if iters is None or iters == float('inf'):
            continue  # Skip unsolvable candidates
            
        candidate_iterations[idx] = iters
    
    return candidate_iterations

if __name__ == "__main__":
    start_time = time.time()
    
    truth_table = np.array([
        [1, 1, 1, 1], [1, 1, 1, 0], [1, 1, 0, 1], [1, 1, 0, 0],
        [1, 0, 1, 1], [1, 0, 1, 0], [1, 0, 0, 1], [1, 0, 0, 0],
        [0, 1, 1, 1], [0, 1, 1, 0], [0, 1, 0, 1], [0, 1, 0, 0],
        [0, 0, 1, 1], [0, 0, 1, 0], [0, 0, 0, 1], [0, 0, 0, 0]
    ])

    found_count = 0
    found_formulas = []

    # Process all rows with progress tracking
    for row_index in range(100):
        print(f"Processing row {row_index}...")
        row = all_rows[row_index]
        final_formula = row[formula_idx]
        bitlist = binary_to_bitlist(row[category_idx], len(truth_table))

        current = None
        for depth in range(10):
            print(f"  Depth {depth}")
            current_options = run_derivation_for_row(row_index, row, column_names, current)
            result = evaluate_candidate_options(current_options, truth_table, final_formula, bitlist)

            if isinstance(result, str):
                found_count += 1
                found_formulas.append((row_index, result))
                print(f"  Found formula: {result}")
                break
            elif result:  # Check if result is not empty
                best_key = min(result, key=result.get)
                current = current_options[best_key]
                print(f"  Best candidate: {tree_to_formula(current)} (iterations: {result[best_key]})")
            else:
                # No viable candidates found
                print("  No viable candidates found")
                break

    print(f"\n=== Summary ===")
    print(f"Found minimal formula in {found_count} cases.")
    for idx, formula in found_formulas:
        print(f" • row {idx}: {formula!r}")

    elapsed = time.time() - start_time
    print(f"Total execution time: {elapsed:.2f} seconds.")

Using device: cuda
Processing row 0...
  Depth 0
Using row 0: (0, 0, 0, 0, 0, 0, 1, 0, 0, 65280, 0, 'p')

Current expression: Z
  Found formula: p
Processing row 1...
  Depth 0
Using row 1: (0, 0, 0, 0, 0, 0, 1, 0, 0, 61680, 0, 'q')

Current expression: Z
  Found formula: q
Processing row 2...
  Depth 0
Using row 2: (0, 0, 0, 0, 0, 0, 1, 0, 0, 52428, 0, 'r')

Current expression: Z
  Found formula: r
Processing row 3...
  Depth 0
Using row 3: (0, 0, 0, 0, 0, 0, 1, 0, 0, 43690, 0, 's')

Current expression: Z
  Found formula: s
Processing row 4...
  Depth 0
Using row 4: (0, 0, 0, 0, 0, 0, 1, 0, 0, 255, 1, 'NA(p,p)')

Current expression: Z
  Best candidate: NA(Z,Z) (iterations: 22)
  Depth 1
Using row 4: (0, 0, 0, 0, 0, 0, 1, 0, 0, 255, 1, 'NA(p,p)')

Current expression: ('NA', 'Z', 'Z')
  Best candidate: NA(p,Z) (iterations: 0)
  Depth 2
Using row 4: (0, 0, 0, 0, 0, 0, 1, 0, 0, 255, 1, 'NA(p,p)')

Current expression: ('NA', 'p', 'Z')
  Found formula: NA(p,p)
Processing row 5...
  Depth 0


  Best candidate: NA(q,NA(q,NA(q,NA(q,Z)))) (iterations: 7)
  Depth 8
Using row 14: (0, 0, 0, 0, 0, 0, 1, 0, 0, 65535, 2, 'NA(p,NA(p,p))')

Current expression: ('NA', 'q', ('NA', 'q', ('NA', 'q', ('NA', 'q', 'Z'))))
  Best candidate: NA(q,NA(q,NA(q,NA(q,NA(Z,Z))))) (iterations: 4)
  Depth 9
Using row 14: (0, 0, 0, 0, 0, 0, 1, 0, 0, 65535, 2, 'NA(p,NA(p,p))')

Current expression: ('NA', 'q', ('NA', 'q', ('NA', 'q', ('NA', 'q', ('NA', 'Z', 'Z')))))
  Best candidate: NA(q,NA(q,NA(q,NA(q,NA(p,Z))))) (iterations: 1)
Processing row 15...
  Depth 0
Using row 15: (0, 0, 0, 0, 0, 0, 1, 0, 0, 61695, 2, 'NA(p,NA(p,q))')

Current expression: Z
  Best candidate: NA(Z,Z) (iterations: 39)
  Depth 1
Using row 15: (0, 0, 0, 0, 0, 0, 1, 0, 0, 61695, 2, 'NA(p,NA(p,q))')

Current expression: ('NA', 'Z', 'Z')
  Best candidate: NA(p,Z) (iterations: 9)
  Depth 2
Using row 15: (0, 0, 0, 0, 0, 0, 1, 0, 0, 61695, 2, 'NA(p,NA(p,q))')

Current expression: ('NA', 'p', 'Z')
  Best candidate: NA(p,NA(Z,Z)) (iteratio

  Best candidate: NA(Z,Z) (iterations: 23)
  Depth 1
Using row 21: (0, 0, 0, 0, 0, 0, 1, 0, 0, 65295, 2, 'NA(q,NA(p,p))')

Current expression: ('NA', 'Z', 'Z')
  Best candidate: NA(q,Z) (iterations: 10)
  Depth 2
Using row 21: (0, 0, 0, 0, 0, 0, 1, 0, 0, 65295, 2, 'NA(q,NA(p,p))')

Current expression: ('NA', 'q', 'Z')
  Best candidate: NA(q,NA(Z,Z)) (iterations: 10)
  Depth 3
Using row 21: (0, 0, 0, 0, 0, 0, 1, 0, 0, 65295, 2, 'NA(q,NA(p,p))')

Current expression: ('NA', 'q', ('NA', 'Z', 'Z'))
  Best candidate: NA(q,NA(p,Z)) (iterations: 0)
  Depth 4
Using row 21: (0, 0, 0, 0, 0, 0, 1, 0, 0, 65295, 2, 'NA(q,NA(p,p))')

Current expression: ('NA', 'q', ('NA', 'p', 'Z'))
  Found formula: NA(q,NA(p,p))
Processing row 22...
  Depth 0
Using row 22: (0, 0, 0, 0, 0, 0, 1, 0, 0, 53007, 2, 'NA(q,NA(p,r))')

Current expression: Z
  Best candidate: NA(Z,Z) (iterations: 20)
  Depth 1
Using row 22: (0, 0, 0, 0, 0, 0, 1, 0, 0, 53007, 2, 'NA(q,NA(p,r))')

Current expression: ('NA', 'Z', 'Z')
  Best ca

  Best candidate: NA(Z,Z) (iterations: 34)
  Depth 1
Using row 26: (0, 0, 0, 0, 0, 0, 1, 0, 0, 36751, 2, 'NA(q,NA(r,s))')

Current expression: ('NA', 'Z', 'Z')
  Best candidate: NA(q,Z) (iterations: 9)
  Depth 2
Using row 26: (0, 0, 0, 0, 0, 0, 1, 0, 0, 36751, 2, 'NA(q,NA(r,s))')

Current expression: ('NA', 'q', 'Z')
  Best candidate: NA(q,NA(Z,Z)) (iterations: 17)
  Depth 3
Using row 26: (0, 0, 0, 0, 0, 0, 1, 0, 0, 36751, 2, 'NA(q,NA(r,s))')

Current expression: ('NA', 'q', ('NA', 'Z', 'Z'))
  Best candidate: NA(q,NA(q,Z)) (iterations: 2)
  Depth 4
Using row 26: (0, 0, 0, 0, 0, 0, 1, 0, 0, 36751, 2, 'NA(q,NA(r,s))')

Current expression: ('NA', 'q', ('NA', 'q', 'Z'))
  Best candidate: NA(q,NA(q,NA(Z,Z))) (iterations: 4)
  Depth 5
Using row 26: (0, 0, 0, 0, 0, 0, 1, 0, 0, 36751, 2, 'NA(q,NA(r,s))')

Current expression: ('NA', 'q', ('NA', 'q', ('NA', 'Z', 'Z')))
  Best candidate: NA(q,NA(q,NA(q,Z))) (iterations: 6)
  Depth 6
Using row 26: (0, 0, 0, 0, 0, 0, 1, 0, 0, 36751, 2, 'NA(q,NA(r,

  Best candidate: NA(r,NA(r,NA(r,NA(q,NA(Z,Z))))) (iterations: 9)
  Depth 9
Using row 31: (0, 0, 0, 0, 0, 0, 1, 0, 0, 46003, 2, 'NA(r,NA(q,s))')

Current expression: ('NA', 'r', ('NA', 'r', ('NA', 'r', ('NA', 'q', ('NA', 'Z', 'Z')))))
  Best candidate: NA(r,NA(r,NA(r,NA(q,NA(q,Z))))) (iterations: 2)
Processing row 32...
  Depth 0
Using row 32: (0, 0, 0, 0, 0, 0, 1, 0, 0, 48059, 2, 'NA(r,NA(r,s))')

Current expression: Z
  Best candidate: NA(Z,Z) (iterations: 10)
  Depth 1
Using row 32: (0, 0, 0, 0, 0, 0, 1, 0, 0, 48059, 2, 'NA(r,NA(r,s))')

Current expression: ('NA', 'Z', 'Z')
  Best candidate: NA(NA(Z,Z),Z) (iterations: 16)
  Depth 2
Using row 32: (0, 0, 0, 0, 0, 0, 1, 0, 0, 48059, 2, 'NA(r,NA(r,s))')

Current expression: ('NA', ('NA', 'Z', 'Z'), 'Z')
  Best candidate: NA(NA(r,Z),Z) (iterations: 18)
  Depth 3
Using row 32: (0, 0, 0, 0, 0, 0, 1, 0, 0, 48059, 2, 'NA(r,NA(r,s))')

Current expression: ('NA', ('NA', 'r', 'Z'), 'Z')
  Best candidate: NA(NA(r,s),Z) (iterations: 10)
  Depth 4

...

  Best candidate: NA(NA(s,Z),Z) (iterations: 8)
  Depth 3
Using row 79: (0, 0, 0, 0, 0, 0, 1, 0, 0, 65520, 3, 'NA(NA(p,p),NA(q,q))')

Current expression: ('NA', ('NA', 's', 'Z'), 'Z')
  Best candidate: NA(NA(s,q),Z) (iterations: 18)
  Depth 4
Using row 79: (0, 0, 0, 0, 0, 0, 1, 0, 0, 65520, 3, 'NA(NA(p,p),NA(q,q))')

Current expression: ('NA', ('NA', 's', 'q'), 'Z')
  Best candidate: NA(NA(s,q),NA(Z,Z)) (iterations: 20)
  Depth 5
Using row 79: (0, 0, 0, 0, 0, 0, 1, 0, 0, 65520, 3, 'NA(NA(p,p),NA(q,q))')

Current expression: ('NA', ('NA', 's', 'q'), ('NA', 'Z', 'Z'))
  Best candidate: NA(NA(s,q),NA(NA(Z,Z),Z)) (iterations: 8)
  Depth 6
Using row 79: (0, 0, 0, 0, 0, 0, 1, 0, 0, 65520, 3, 'NA(NA(p,p),NA(q,q))')

Current expression: ('NA', ('NA', 's', 'q'), ('NA', ('NA', 'Z', 'Z'), 'Z'))
  Best candidate: NA(NA(s,q),NA(NA(p,Z),Z)) (iterations: 13)
  Depth 7
Using row 79: (0, 0, 0, 0, 0, 0, 1, 0, 0, 65520, 3, 'NA(NA(p,p),NA(q,q))')

Current expression: ('NA', ('NA', 's', 'q'), ('NA', ('NA',

  Best candidate: NA(NA(p,p),NA(NA(p,Z),Z)) (iterations: 5)
  Depth 7
Using row 83: (0, 0, 0, 0, 0, 0, 1, 0, 0, 65416, 3, 'NA(NA(p,p),NA(r,s))')

Current expression: ('NA', ('NA', 'p', 'p'), ('NA', ('NA', 'p', 'Z'), 'Z'))
  Best candidate: NA(NA(p,p),NA(NA(p,p),Z)) (iterations: 8)
  Depth 8
Using row 83: (0, 0, 0, 0, 0, 0, 1, 0, 0, 65416, 3, 'NA(NA(p,p),NA(r,s))')

Current expression: ('NA', ('NA', 'p', 'p'), ('NA', ('NA', 'p', 'p'), 'Z'))
  Best candidate: NA(NA(p,p),NA(NA(p,p),NA(Z,Z))) (iterations: 7)
  Depth 9
Using row 83: (0, 0, 0, 0, 0, 0, 1, 0, 0, 65416, 3, 'NA(NA(p,p),NA(r,s))')

Current expression: ('NA', ('NA', 'p', 'p'), ('NA', ('NA', 'p', 'p'), ('NA', 'Z', 'Z')))
  Best candidate: NA(NA(p,p),NA(NA(p,p),NA(NA(Z,Z),Z))) (iterations: 20)
Processing row 84...
  Depth 0
Using row 84: (0, 0, 0, 0, 0, 0, 1, 0, 0, 65450, 3, 'NA(NA(p,p),NA(s,s))')

Current expression: Z
  Best candidate: NA(Z,Z) (iterations: 8)
  Depth 1
Using row 84: (0, 0, 0, 0, 0, 0, 1, 0, 0, 65450, 3, 'NA(NA(p,

  Best candidate: NA(NA(p,q),NA(s,NA(s,NA(p,Z)))) (iterations: 0)
Processing row 88...
  Depth 0
Using row 88: (0, 0, 0, 0, 0, 0, 1, 0, 0, 61632, 3, 'NA(NA(p,q),NA(q,r))')

Current expression: Z
  Best candidate: NA(Z,Z) (iterations: 17)
  Depth 1
Using row 88: (0, 0, 0, 0, 0, 0, 1, 0, 0, 61632, 3, 'NA(NA(p,q),NA(q,r))')

Current expression: ('NA', 'Z', 'Z')
  Best candidate: NA(NA(Z,Z),Z) (iterations: 14)
  Depth 2
Using row 88: (0, 0, 0, 0, 0, 0, 1, 0, 0, 61632, 3, 'NA(NA(p,q),NA(q,r))')

Current expression: ('NA', ('NA', 'Z', 'Z'), 'Z')
  Best candidate: NA(NA(p,Z),Z) (iterations: 4)
  Depth 3
Using row 88: (0, 0, 0, 0, 0, 0, 1, 0, 0, 61632, 3, 'NA(NA(p,q),NA(q,r))')

Current expression: ('NA', ('NA', 'p', 'Z'), 'Z')
  Best candidate: NA(NA(p,q),Z) (iterations: 30)
  Depth 4
Using row 88: (0, 0, 0, 0, 0, 0, 1, 0, 0, 61632, 3, 'NA(NA(p,q),NA(q,r))')

Current expression: ('NA', ('NA', 'p', 'q'), 'Z')
  Best candidate: NA(NA(p,q),NA(Z,Z)) (iterations: 13)
  Depth 5
Using row 88: (0, 0,

  Best candidate: NA(NA(r,p),Z) (iterations: 7)
  Depth 4
Using row 93: (0, 0, 0, 0, 0, 0, 1, 0, 0, 52224, 3, 'NA(NA(p,r),NA(p,r))')

Current expression: ('NA', ('NA', 'r', 'p'), 'Z')
  Best candidate: NA(NA(r,p),NA(Z,Z)) (iterations: 6)
  Depth 5
Using row 93: (0, 0, 0, 0, 0, 0, 1, 0, 0, 52224, 3, 'NA(NA(p,r),NA(p,r))')

Current expression: ('NA', ('NA', 'r', 'p'), ('NA', 'Z', 'Z'))
  Best candidate: NA(NA(r,p),NA(p,Z)) (iterations: 2)
  Depth 6
Using row 93: (0, 0, 0, 0, 0, 0, 1, 0, 0, 52224, 3, 'NA(NA(p,r),NA(p,r))')

Current expression: ('NA', ('NA', 'r', 'p'), ('NA', 'p', 'Z'))
  Best candidate: NA(NA(r,p),NA(p,NA(Z,Z))) (iterations: 5)
  Depth 7
Using row 93: (0, 0, 0, 0, 0, 0, 1, 0, 0, 52224, 3, 'NA(NA(p,r),NA(p,r))')

Current expression: ('NA', ('NA', 'r', 'p'), ('NA', 'p', ('NA', 'Z', 'Z')))
  Best candidate: NA(NA(r,p),NA(p,NA(p,Z))) (iterations: 3)
  Depth 8
Using row 93: (0, 0, 0, 0, 0, 0, 1, 0, 0, 52224, 3, 'NA(NA(p,r),NA(p,r))')

Current expression: ('NA', ('NA', 'r', 'p'

  Best candidate: NA(NA(Z,Z),Z) (iterations: 10)
  Depth 2
Using row 98: (0, 0, 0, 0, 0, 0, 1, 0, 0, 52360, 3, 'NA(NA(p,r),NA(r,s))')

Current expression: ('NA', ('NA', 'Z', 'Z'), 'Z')
  Best candidate: NA(NA(s,Z),Z) (iterations: 14)
  Depth 3
Using row 98: (0, 0, 0, 0, 0, 0, 1, 0, 0, 52360, 3, 'NA(NA(p,r),NA(r,s))')

Current expression: ('NA', ('NA', 's', 'Z'), 'Z')
  Best candidate: NA(NA(s,NA(Z,Z)),Z) (iterations: 11)
  Depth 4
Using row 98: (0, 0, 0, 0, 0, 0, 1, 0, 0, 52360, 3, 'NA(NA(p,r),NA(r,s))')

Current expression: ('NA', ('NA', 's', ('NA', 'Z', 'Z')), 'Z')
  Best candidate: NA(NA(s,NA(s,Z)),Z) (iterations: 15)
  Depth 5
Using row 98: (0, 0, 0, 0, 0, 0, 1, 0, 0, 52360, 3, 'NA(NA(p,r),NA(r,s))')

Current expression: ('NA', ('NA', 's', ('NA', 's', 'Z')), 'Z')
  Best candidate: NA(NA(s,NA(s,s)),Z) (iterations: 7)
  Depth 6
Using row 98: (0, 0, 0, 0, 0, 0, 1, 0, 0, 52360, 3, 'NA(NA(p,r),NA(r,s))')

Current expression: ('NA', ('NA', 's', ('NA', 's', 's')), 'Z')
  Best candidate: N